In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer

import nltk

import shap

# Load data

In [ ]:
df = pd.read_csv('data/Sentences_clean.csv')

In [ ]:
df.head()

In [ ]:
# create stratified train test split 

train, test = train_test_split(df, test_size=0.2, train_size=0.8, stratify=df['sentiment'], random_state=0)

In [ ]:
# encode target

le = LabelEncoder()

le.fit(train['sentiment'])

train['target'] = le.transform(train['sentiment'])
test['target'] = le.transform(test['sentiment'])

In [ ]:
le.classes_

# Create features

In [ ]:
# create bag of words

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Sentence_clean'])
test_matrix = vectorizer.transform(test['Sentence_clean'])

# Train simple logistic regression model

In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['target']
y_test = test['target']

In [ ]:
# logistic regression model

lr = LogisticRegression(random_state=0)

# fit
lr.fit(X_train,y_train)

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
# plot confusions matrix

cf_matrix = confusion_matrix(predictions,y_test)

disp = ConfusionMatrixDisplay(cf_matrix, display_labels=le.classes_)

disp.plot()

plt.show()

In [ ]:
print(classification_report(predictions,y_test, labels=[0,1,2], target_names=le.classes_))

# Explain sentiment with SHAP

In [ ]:
# initialize for plotting
shap.initjs()

In [ ]:
# create linear SHAP explainer

explainer = shap.explainers.Linear(lr, X_train.toarray(),output_names=le.classes_, feature_names=vectorizer.get_feature_names_out())

# calculate SHAP values
shap_values = explainer.shap_values(X_test.toarray())

In [ ]:
# create SHAP summary plot 
shap.summary_plot(shap_values, X_test.toarray(), plot_type="bar", class_names= le.classes_, feature_names = vectorizer.get_feature_names_out(), color=plt.get_cmap("tab20c"))

In [ ]:
# create SHAP force plot for positive instance

idx = 1
# only explain prediction for top class
c = predictions[idx]

print('Force plot for test sample with index {} classified as {}'.format(idx, le.classes_[c].lower()))

shap.force_plot(explainer.expected_value[c], shap_values[c][idx,:], X_test.toarray()[idx], feature_names = vectorizer.get_feature_names_out())

In [ ]:
# create SHAP force plot for neutral instance
idx = 0
# only explain prediction for top class
c = predictions[idx]

print('Force plot for test sample with index {} classified as {}'.format(idx, le.classes_[c].lower()))

shap.force_plot(explainer.expected_value[c], shap_values[c][idx,:], X_test.toarray()[idx], feature_names = vectorizer.get_feature_names_out())

In [ ]:
# create SHAP force plot for negative instance
idx = -5
# only explain prediction for top class
c = predictions[idx]

print('Force plot for test sample with index {} classified as {}'.format(idx, le.classes_[c].lower()))

shap.force_plot(explainer.expected_value[c], shap_values[c][idx,:], X_test.toarray()[idx], feature_names = vectorizer.get_feature_names_out())